[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/integrations/haystack/extractive-qa/haystack_extractive_qa.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/integrations/haystack/extractive-qa/haystack_extractive_qa.ipynb)

# Extractive Question Answering in Haystack

This notebook demonstrates how you can build an Extractive Question-Answering (QA) system with Pinecone and Haystack. We need three main components to build an Extractive QA in Haystack: DocumentStore, Retriever and Reader. 

## Document Store

The document store is where all our documents are stored. Haystack has different document stores we can use for various use cases. We will use a dense / embedding-based retriever so, we need a vector-optimized document store to hold embedding vectors that represent our documents. We will use the `PineconeDocumentStore` which helps to store our documents and embeddings in the Pinecone Vector Database and efficiently perform similarity search.

# Retriever

We will use Haystack’s `EmbeddingRetriever` in our Extractive QA pipeline. It generates the document embeddings stored in the document store and the query embeddings needed to perform similarity search. The QA system needs information relevant to the query to extract answers. We need to retrieve the documents containing relevant answers from the document store. The retriever’s job is to find the best candidates by computing the similarity between the question and the document embeddings. The final answer is generated based on the best candidates.

# Reader

The Reader takes a question and a set of Documents as input and returns an answer by selecting a text span within the Documents. 

# Install Dependencies

In [ ]:
!pip install -U farm-haystack>=1.3.0 pinecone-client datasets

# Load the Dataset

We will use the SQuAD dataset, which consists of questions and context paragraphs containing question answers. Let's load the SQUAD dataset from the HuggingFace Model Hub. We load the dataset into a pandas dataframe and filter the title, question, and context columns, and we drop any duplicate context passages.

In [1]:
from datasets import load_dataset

# load the squad dataset into a pandas dataframe
df = load_dataset("squad", split="train").to_pandas()
# select only title and context column
df = df[["title", "context"]]
# drop rows containing duplicate context passages
df = df.drop_duplicates(subset="context")
df.head()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /Users/jamesbriggs/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


,title,context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
5,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
10,University_of_Notre_Dame,The university is the major seat of the Congre...
15,University_of_Notre_Dame,The College of Engineering was established in ...
20,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...


# Initializing the DocumentStore

We need an API key to use the PineconeDocumentStore in Haystack (you can sign up for free [here](https://app.pinecone.io/) and get an API key). 

In [2]:
from haystack.document_stores import PineconeDocumentStore

document_store = PineconeDocumentStore(
    api_key='<<YOUR_API_KEY>>',
    index='haystack-extractive-qa',
    similarity="cosine",
    embedding_dim=384
)

INFO - haystack.document_stores.pinecone -  Index statistics: name: haystack-extractive-qa, embedding dimensions: 384, record count: 0


The above code is all we need to initialize a connection to a Pinecone index in Haystack. It will either create a Pinecone index named "haystack-extractive-qa" if it does not exisit or connect to an existing index with the same name. We specify the similarity type as "cosine" and dimension as 384 because the retriever we use is optimized for cosine similarity and outputs 384-dimension vectors.

# Initialze Retriever


We will use Haystack's `EmbeddingRetriever` with a SentenceTransformer model which has been designed for semantic search. This model has been trained on 215M (question, answer) pairs from diverse sources and performs quite well for comparing the similarity between queries and documents. We can use the retriever to easily compute and update the embeddings for all the documents in the document store.

In [3]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="multi-qa-MiniLM-L6-cos-v1",
    model_format="sentence_transformers"
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.retriever.dense -  Init retriever using embeddings of model multi-qa-MiniLM-L6-cos-v1


Let's upsert the documents to the Pinecone index.

In [4]:
from haystack import Document

docs = []
for d in df.iterrows():
    d = d[1]
    # create haystack document object with text content and doc metadata
    doc = Document(
        content=d["context"],
        meta={
            "title": d["title"],
            'context': d['context']
        }
    )
    docs.append(doc)

# upsert the data document to pinecone index
document_store.write_documents(docs)

Writing Documents:   0%|          | 0/18891 [00:00<?, ?it/s]

The context passages are now in Pinecone index. Let's generate embeddings for our context passages using the retriever. All we need to do is pass the retriever to `update_embeddings` method in the document store. This will generate embeddings and upsert it to Pinecone Index.

In [5]:
document_store.update_embeddings(
    retriever,
    batch_size=128
)

INFO - haystack.document_stores.pinecone -  Updating embeddings for 18891 docs...


Updating Embedding:   0%|          | 0/18891 [00:00<?, ? docs/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

The embeddings are updated to the document store, and our retriever is now ready. Let's test the retriever before we use it in the Extractive QA pipeline. We can test queries with the retriever by loading it into a `DocumentSearchPipeline`.

In [6]:
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

# load retriever into a documnet search pipeline
search_pipe = DocumentSearchPipeline(retriever)

In [7]:
result = search_pipe.run(
    query="What was Albert Einstein famous for?",
    params={"Retriever": {"top_k": 2}}
)

print_documents(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: What was Albert Einstein famous for?

{   'content': 'Albert Einstein is known for his theories of special '
               'relativity and general relativity. He also made important '
               'contributions to statistical mechanics, especially his '
               'mathematical treatment of Brownian motion, his resolution of '
               'the paradox of specific heats, and his connection of '
               'fluctuations and dissipation. Despite his reservations about '
               'its interpretation, Einstein also made contributions to '
               'quantum mechanics and, indirectly, quantum field theory, '
               'primarily through his theoretical studies of the photon.',
    'name': None}

{   'content': 'Albert Einstein lived in a flat at the Kramgasse 49, the site '
               'of the Einsteinhaus, from 1903 to 1905, the year in which the '
               'Annus Mirabilis Papers were published.',
    'name': None}



As you can see, the retriever can find relevant documents from the document store. Now let's initialize the Reader.

# Initialize Reader

We use the `deepset/electra-base-squad2` model from the HuggingFace model hub as our reader model. We load this model into an extractive QA pipeline from Haystack. Under the hood it will read our questions and context passages individually and extract smaller answer snippets, returning the most relevant to us.

In [8]:
from haystack.nodes import FARMReader

reader = FARMReader(
    model_name_or_path='deepset/electra-base-squad2', 
    use_gpu=True
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/electra-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/electra-base-squad2
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

INFO - haystack.modeling.infer -  Got ya 9 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\  /w\ 
INFO - haystack.modeling.infer -  /'\   / \   /'\   /'\   / \   / \   /'\   /'\   /'\ 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Initialize Extractive QA Pipelin

The extractive QA pipeline combines all of what we have initialized already into one easy-to-use pipeline.

In [9]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

Now all the components we need are ready. Let's write a helper function and execute queries.



# Querying

To query we can now run the `pipe.run` method.

In [10]:
from haystack.utils import print_answers

query = "What was Albert Einstein famous for?"
# get the answer
answer = pipe.run(
    query=query,
    params={
        "Retriever": {"top_k": 1},
    }
)
# print the answer(s)
print_answers(answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.53 Batches/s]


Query: What was Albert Einstein famous for?
Answers:
[   <Answer {'answer': 'his theories of special relativity and general relativity', 'type': 'extractive', 'score': 0.993550717830658, 'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics,', 'offsets_in_document': [{'start': 29, 'end': 86}], 'offsets_in_context': [{'start': 29, 'end': 86}], 'document_id': '23357c05e3e46bacea556705de1ea6a5', 'meta': {'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics, especially his mathematical treatment of Brownian motion, his resolution of the paradox of specific heats, and his connection of fluctuations and dissipation. Despite his reservations about its interpretation, Einstein also made contributions to quantum mechanics and, indirectly, quantum field theory, primarily through hi

In [11]:
question = "How much oil is Egypt producing in a day?"
# get the answer
answer = pipe.run(
    query=query,
    params={
        "Retriever": {"top_k": 1},
    }
)
# print the answer(s)
print_answers(answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.11 Batches/s]


Query: What was Albert Einstein famous for?
Answers:
[   <Answer {'answer': 'his theories of special relativity and general relativity', 'type': 'extractive', 'score': 0.993550717830658, 'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics,', 'offsets_in_document': [{'start': 29, 'end': 86}], 'offsets_in_context': [{'start': 29, 'end': 86}], 'document_id': '23357c05e3e46bacea556705de1ea6a5', 'meta': {'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics, especially his mathematical treatment of Brownian motion, his resolution of the paradox of specific heats, and his connection of fluctuations and dissipation. Despite his reservations about its interpretation, Einstein also made contributions to quantum mechanics and, indirectly, quantum field theory, primarily through hi

In [12]:
question = "What are the first names of the men that invented youtube?"
# get the answer
answer = pipe.run(
    query=query,
    params={
        "Retriever": {"top_k": 1},
    }
)
# print the answer(s)
print_answers(answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.13 Batches/s]


Query: What was Albert Einstein famous for?
Answers:
[   <Answer {'answer': 'his theories of special relativity and general relativity', 'type': 'extractive', 'score': 0.993550717830658, 'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics,', 'offsets_in_document': [{'start': 29, 'end': 86}], 'offsets_in_context': [{'start': 29, 'end': 86}], 'document_id': '23357c05e3e46bacea556705de1ea6a5', 'meta': {'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics, especially his mathematical treatment of Brownian motion, his resolution of the paradox of specific heats, and his connection of fluctuations and dissipation. Despite his reservations about its interpretation, Einstein also made contributions to quantum mechanics and, indirectly, quantum field theory, primarily through hi

We can return multiple answers by setting the `top_k` parameter.

In [13]:
question = "Who was the first person to step foot on the moon?"
# get the answer
answer = pipe.run(
    query=query,
    params={
        "Retriever": {"top_k": 3},
    }
)
# print the answer(s)
print_answers(answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.86 Batches/s]


Query: What was Albert Einstein famous for?
Answers:
[   <Answer {'answer': 'his theories of special relativity and general relativity', 'type': 'extractive', 'score': 0.993550717830658, 'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics,', 'offsets_in_document': [{'start': 29, 'end': 86}], 'offsets_in_context': [{'start': 29, 'end': 86}], 'document_id': '23357c05e3e46bacea556705de1ea6a5', 'meta': {'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics, especially his mathematical treatment of Brownian motion, his resolution of the paradox of specific heats, and his connection of fluctuations and dissipation. Despite his reservations about its interpretation, Einstein also made contributions to quantum mechanics and, indirectly, quantum field theory, primarily through hi

---